In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dask.dataframe as dd

c:\sourceee\Energy-prediction\.venv\Lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:

# Load the datasets
#client = pd.read_csv('client.csv')
#gs_prices = pd.read_csv('gas_prices.csv')
#electricity_prices = pd.read_csv('electricity_prices.csv')
train = dd.read_csv('C:/sourceee/Energy-Prediction/predict-energy-behavior-of-prosumers/train.csv')
#historical_weather = pd.read_csv('historical_weather.csv')
forecast_weather = dd.read_csv('C:/sourceee/Energy-Prediction/predict-energy-behavior-of-prosumers/forecast_weather.csv')
#weather_station_to_county = pd.read_csv('weather_station_to_county_mapping.csv')

In [3]:
# Merge train data with forecast weather data
merged_train_forecast = dd.merge(train, forecast_weather, how='left', left_on=['data_block_id', 'datetime'], right_on=['data_block_id', 'forecast_datetime'])


In [4]:
# Convert the DataFrame to a Dask DataFrame with chunking
dask_df = dd.from_pandas(merged_train_forecast, npartitions=10)

# Define features and target columns
features = [...]  # List of feature column names
target = 'target'  # Name of the target column

# Compute the correlation matrix using Dask operations
corr_matrix = dask_df[features + [target]].corr().compute()

# Plot the heatmap using Seaborn
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix of Forecast Weather Features and Target Variable (Dask Data)')
plt.show()

MemoryError: Unable to allocate 1.14 GiB for an array with shape (8, 19206106) and data type float64

In [7]:

# Define chunk size to reduce memory load during conversion
chunk_size = 5000  # Adjust as needed based on your dataset size

# Convert pandas DataFrame to Dask DataFrame with specified chunk size
dask_df = dd.from_pandas(merged_train_forecast, chunksize=chunk_size)


MemoryError: 